# Space Invaders - des images et une meute d'envahisseurs

## Gestion des images

Nous utiliserons les images suivantes; vous pouvez les récupérer dans le dossier *images*:

|                *hero.png*                |          *env_mere.png*          |          *env_fils.png*          |
|:----------------------------------------:|:--------------------------------:|:--------------------------------:|
| <img src="images/hero.png"/>             | <img src="images/env_mere.png"/> | <img src="images/env_fils.png"/> |

Commencer par déposer ces images dans un répertoire (à créer) `images` situé dans le même répertoire que vos fichier *....py*.

- `Acteur`: modifions la fonction `__init__` afin de pouvoir afficher une image à la place d'un rectangle:
```python
def __init__(self, scene, largeur=100, hauteur=100, couleur="white", image=None):
    # ...
    self.image = image
    if image is not None:
        # tenir compte des dimensions de l'image éventuelle
        self.dim = image.width(), image.height()
    else:
        self.dim = largeur, hauteur
    #...
```

- Adaptons la fonction `creer_acteur` de la `Scene`:
```python
def creer_acteur(self, acteur):
    xmin, ymin, xmax, ymax = acteur.min_max()
    if acteur.image is not None:
        # on «ancre l'image» au nord ouest (coin supérieur gauche)
        # car elle est ancrée au centre par défaut (voir doc).
        _id = __.create_image(__, __, image=acteur.__, anchor="nw")
    else:

        _id = __.__(xmin, ymin, xmax, ymax, fill=__, width=0)
    # mémorisons cet acteur
    __(acteur)
    return _id
```

**Solution**

```python
def creer_acteur(self, acteur):
    xmin, ymin, xmax, ymax = acteur.min_max()
    if acteur.image is not None:
        # on «ancre l'image» au nord ouest (coin supérieur gauche)
        # car elle est ancrée au centre par défaut (voir doc).
        _id = self.create_image(xmin, ymin, image=acteur.image, anchor="nw")
    else:

        _id = self.create_rectangle(xmin, ymin, xmax, ymax, fill=acteur.couleur, width=0)
    # mémorisons cet acteur
    self.acteurs.add(acteur)
    return _id
```

- Enfin, testons avec `Hero` en modifiant son initialisateur:
```python
def __init__(self, scene):
        super().__init__(scene, image=PhotoImage(file="img/hero.png"))
        
        #...
```
Ne pas oublier d'importer `PhotoImage`: `from tkinter import PhotoImage`

### Exercice

Adapter de la même façon l'envahisseur

**Solution**

```python
#...
from tkinter import PhotoImage

class Envahisseur(Acteur):
    
    def __init__(self, scene):
        super().__init__(scene, image=PhotoImage(file="images/env_mere.png"))
        #...
```

## La meute des envahisseurs

Les envahisseurs «enfants» se déplacent en formation de type grille. Ils balaient l'écran horizontalement puis ils se dirigent vers le bas lorsqu'ils atteignent le bord de la scène.

Pour les gérer, nous allons utiliser un **tableau**.

### `Tableau`

Pour maîtriser nos envahisseurs en meute, nous avons besoin d'un tableau. Nous prendrons les conventions suivantes (voir illustration plus loin):

- `i` désigne un numéro de ligne, il est situé entre `0` et `nb_lignes`,
- `j` désigne un numéro de colonne, il est situé entre `0` et `nb_colonnes`,
- `None` représente l'absence d'une valeur,
- `jmin` et `jmax` représentent respectivement le numéro de la première et dernière colonne non vide,
- `frontiere_basse` désigne la liste des valeurs situées tout à la fin de chaque colonne (de gauche à droite) - voir croix rouge.

<img src="illustrations/Tableaux.png"/>

Python ne dispose pas de type «tableau», mais on peut *représenter* un tel tableau avec une liste de listes, chaque *sous-liste* représentant une **ligne** du tableau. 

Par exemple, le tableau précédent s'écrirais:
```python
[
    [None,  'X',  'X',  'X', None],  # t[0]
    [ 'X', None,  'X', None, None],  # t[1]
    [' X', None, None, None, None],  # t[2]
]
```

Si `t` désigne ce tableau alors `t[0]`, `t[1]`, `t[2]` font respectivement références aux lignes n°0, 1, 2.

- `t[i][j]`: lecture `get` de l'élément situé en ligne n°`i` et en colonne n°`j`,

- `t[i][j] = valeur`: écriture de `valeur` dans la case dont la ligne est `i` et la colonne `j`,

- Pour **parcourir** l'ensemble des valeurs d'un tel tableau on utilise une «double boucle»:

  ```python
  for i in range(nb_lignes):
      for j in range(nb_colonnes):
          v = t[i][j]
          # faire quelquechose avec v.
  ```

### Interface

Pour faciliter les choses, nous allons définir une classe `Tableau` munie de l'interface suivante:

```python
class Tableau:
    def __init__(self, nb_lignes, nb_colonnes):
        self.nb_lignes = nb_lignes
        self.nb_colonnes = nb_colonnes
        self.nb_valeurs = 0

        # amorçage de la liste de liste qui représente le tableau
        tableau = []
        for __ in range(nb_lignes):
            # construction de la ligne __
            ligne = []
            for __ in range(nb_colonnes):
                ligne.__(None)

            # ajout de la ligne au tableau
            __(ligne)

        self.tableau = tableau
    
    def ajouter(self, valeur, i, j):
        "ajoute valeur dans la case i,j si elle est vide"
        assert 0 <= i < self.nb_lignes and 0 <= j < self.nb_colonnes
        pass
    
    def supprimer(self, i, j):
        "supprime la valeur de la case i,j"
        assert 0 <= i < self.nb_lignes and 0 <= j < self.nb_colonnes
        pass
    
    def supprimer_valeur(self, valeur):
        "supprimer valeur du tableau si elle y figure"
        pass
    
    def ligne(self, i):
        "renvoie la ligne i"
        assert 0 <= i < self.nb_lignes
        return self.tableau[i]
    
    def colonne(self, j):
        "renvoie une liste qui correspond à la colonne j"
        assert 0 <= j < self.nb_colonnes
        pass
    
    def liste_valeurs(self):
        "renvoie la liste des valeurs différente de None"
        pass
    
    def frontiere_basse(self):
        "renvoie la liste des valeurs qui sont les dernières de leur colonne"
        pass
    
    def jmin_max(self):
        "renvoie les numéros des colonnes frontières gauche et droite"
        pass
    
    def est_vide(self):
        return self.nb_valeurs == 0       
```

**Solution** pour `__init__`

```python
def __init__(self, nb_lignes, nb_colonnes):
    self.nb_lignes = nb_lignes
    self.nb_colonnes = nb_colonnes
    self.nb_valeurs = 0

    # amorçage de la liste de liste qui représente le tableau
    tableau = []
    for i in range(nb_lignes):
        # construction de la ligne i
        ligne = []
        for j in range(nb_colonnes):
            ligne.append(None)

        # ajout de la ligne au tableau
        tableau.append(ligne)

    self.tableau = tableau
```

### `ajouter` et `supprimer`

```python
def ajouter(self, valeur, i, j):
    "ajoute valeur dans la case i,j si elle est vide"
    assert 0 <= __ < self.nb_lignes and 0 <= j < __
    
    if self.tableau[i][j] is not __: return
    
    self.tableau[i][j] = __
    self.nb_valeurs __

def supprimer(self, i, j):
    "supprime la valeur de la case i,j"
    assert 0 <= __ < self.nb_lignes and 0 <= j < __
    self.tableau[i][j] = __
    self.nb_valeurs __
```

**Solution**

```python
def ajouter(self, valeur, i, j):
    "ajoute valeur dans la case i,j si elle est vide"
    assert 0 <= i < self.nb_lignes and 0 <= j < self.nb_colonnes
    
    if self.tableau[i][j] is not None: return
    
    self.tableau[i][j] = valeur
    self.nb_valeurs += 1

def supprimer(self, i, j):
    "supprime la valeur de la case i,j"
    assert 0 <= i < self.nb_lignes and 0 <= j < self.nb_colonnes
    self.tableau[i][j] = None
    self.nb_valeurs -=1
```

### `supprimer_valeur`

```python
def supprimer_valeur(self, valeur):
    "supprimer valeur du tableau autant de fois qu'elle y figure"
    a_supprimer = []
    for i in range(self.nb_lignes):
        for j in range(self.nb_colonnes):
            v = self.tableau[i][j]
            if valeur is __ or valeur == __:
                __( (i, j) )
    
    for i, j in a_supprimer:
        self.__(i, j)
```

**Solution**

```python
def supprimer_valeur(self, valeur):
    "supprimer valeur du tableau autant de fois qu'elle y figure"
    a_supprimer = []
    for i in range(self.nb_lignes):
        for j in range(self.nb_colonnes):
            v = self.tableau[i][j]
            if valeur is v or valeur == v:
                a_supprimer.append( (i, j) )
    
    for i, j in a_supprimer:
        self.supprimer(i, j)
```

### `colonne` et `liste_valeurs`

```python
def colonne(self, j):
    "renvoie une liste qui correspond à la colonne j"
    assert 0 <= __ < self.nb_colonnes
    
    col = []
    for i in range(__):
        col.__(__[i][j])
    
    return col

def liste_valeurs(self):
    "renvoie la liste des valeurs différente de None"
    valeurs = []
    for i in range(__):
        for j in range(__):
            v = self.__[i][j]
            if v is not None:
                __(v)
    
    return valeurs
```

**Solution**

```python
def colonne(self, j):
    "renvoie une liste qui correspond à la colonne j"
    assert 0 <= j < self.nb_colonnes
    
    col = []
    for i in range(self.nb_lignes):
        col.append(self.tableau[i][j])
    
    return col

def liste_valeurs(self):
    "renvoie la liste des valeurs différente de None"
    valeurs = []
    for i in range(self.nb_lignes):
        for j in range(self.nb_colonnes):
            v = self.tableau[i][j]
            if v is not None:
                valeurs.append(v)
    
    return valeurs
```

### `frontiere_basse`

Celle-ci est plus difficile: il nous faut trouver la dernière valeur non nulle de chaque colonne.

#### Une fonction auxiliaire

Écrivons une fonction **auxiliaire** (on pourra la placer juste avant la classe `Tableau`) `index_derniere_valeur(liste)` qui renvoie l'index de la dernière valeur non nulle de la liste fournie en argument ou -1 si il n'y a aucune valeur non nulle.

```python
def index_derniere_valeur(liste):
    N = len(liste)
    i = __ # index du dernier élément de liste
    
    while i __ and liste[i] == None:
        # reculons
        i -= 1
    
    return __
```

**Solution**

```python
def index_derniere_valeur(liste):
    N = len(liste)
    i = N - 1 # index du dernier élément de liste
    
    while i >= 0 and liste[i] == None:
        # reculons
        i -= 1
    
    return i
```

#### Exercice

Écrire la fonction similaire `index_premiere_valeur(liste)`

**Solution**

```python
def index_premiere_valeur(liste):
    N = len(liste)
    i = 0 # index du premier élément de liste
    
    while i < N and liste[i] == None:
        # avançons
        i += 1
    
    return i
```

#### Implémentation

Implémentons maintenant `frontiere_basse`

```python
def frontiere_basse(self):
    "renvoie la liste des valeurs qui sont les dernières de leur colonne"
    fr_basse = []
    for j in range(__):
        col = self.__(j)
        index = index_derniere_valeur(__)
        if index >= 0:
            v = col[__]
            fr_basse.__(v)
    
    return fr_basse
```

**Solution**

```python
def frontiere_basse(self):
    "renvoie la liste des valeurs qui sont les dernières de leur colonne"
    fr_basse = []
    for j in range(self.nb_colonnes):
        col = self.colonne(j)
        index = index_derniere_valeur(col)
        if index >= 0:
            v = col[index]
            fr_basse.append(v)
    
    return fr_basse
```

## Envahisseurs

À ce stade, il nous faut distinguer les envahisseurs; nous appelerons:
- `EnvahisseurMere`: celui qui se déplace en haut de l'écran de gauche à droite
- `EnvahisseurFils`: ceux qui se déplacent en meute horizontalement et vers le bas.


Il n'y a rien de bien nouveau dans cette partie sauf que:

- Un `EnvahisseurFils` est construit sur la base de la scène et aussi de la **meute** dont il fait partie.
- Un `EnvahisseurMere` ne tire pas: on a donc supprimé le code correspondant.

Remplacer le contenu du fichier *envahisseurs.py* par:

```python
from acteur import Acteur
from missile import Missile
from random import choice
from tkinter import PhotoImage

class EnvahisseurMere(Acteur):
    
    def __init__(self, scene):
        super().__init__(scene, image=PhotoImage(file="images/env_mere.png"))
        # choisir une vitesse au hasard
        self.v = choice([-100, 100])
        self._annul = None
        
        # régler la position de façon à apparaîte en haut
        # à gauche ou à droite selon la vitesse ...
        l, _ = self.dim
        if self.v > 0:
            self.set_position(-l+10, 30)
        else:
            self.set_position(self.L-10, 30)
        
        # tout est prêt! lançons le.
        self._lancer()

    def supprimer(self):
        super().supprimer()
        self.scene.after_cancel(self._annul)

    def _lancer(self, duree_milli=50):
        if self.est_hors_scene():
            self.supprimer()
            return
        
        self.deplacer_duree(duree_milli / 1000)
        
        self._annul = self.scene.after(
            duree_milli,
            lambda: self._lancer(duree_milli),
        )
```

```python
class EnvahisseurFils(Acteur):
    
    def __init__(self, scene, meute):
        super().__init__(scene, image=PhotoImage(file="images/env_fils.png"))
        self.meute = meute
    
    def tirer(self):
        x, y = self.pos
        l, h = self.dim
        m = Missile(self.scene)
        m.set_position(x + l//2, y + h)
        m.set_vitesse(300)
        m.lancer()

if __name__ == "__main__":
    from scene import *
    from hero import Hero
    EnvahisseurMere(scene)
    Hero(scene)
    fen.mainloop()

```

## La `Meute` - du tableau à la grille

Ici, le **tableau** est une structure *logique* (pas de représentation graphique).

Lorsqu'on souhaite représenter (dessiner) un tableau, nous parlerons de **grille** laquelle est constituée de **cases**.

Chaque **case** possède des *coordonnées* $(x;y)$ pour son coin supérieur gauche et des *dimensions* (largeur et hauteur).

Pour dessiner la **grille** qui correspond au *tableau sous-jacent*, il faut associer aux coordonnées $(i;j)$ du tableau, les coordonnées $(x;y)$ pour la case correspondante.

Créer un nouveau fichier *meute.py* et pour bien comprendre le code de cette classe, étudier soigneusement le passage du tableau à la représentation graphique (grille):

<img src="illustrations/tableau_vers_grille.png"/>

Ainsi qu'au problème du centrage d'un rectangle (l'envahisseur) dans un autre (la case):

<img src="illustrations/rectangle_centre_dans_un_autre.png"/>

Voici l'interface la classe `Meute`:

```python
from tableau import Tableau
from envahisseurs import EnvahisseurFils
from random import choice, randint

class Meute:
    def __init__(self, scene, nb_lignes=3, nb_colonnes=5, largeur_case=70, hauteur_case=70):
        # attributs généraux
        self.scene = scene
        self.nb_lignes = nb_lignes
        self.nb_colonnes = nb_colonnes
        self.largeur_case = largeur_case
        self.hauteur_case = hauteur_case
        
        self.tableau = Tableau(nb_lignes, nb_colonnes)
        self.pos = 0, 0 # position globale de la meute
        
        # gestion du déplacement
        self.pas = 20 # en pixel
        self.delai = 500 # durée entre deux pas en milliseconde
        
        # remplissons les cases de la meute.
        # la position de cette meute est x=0 et y=0 (coin supérieur gauche)
        for i in range(nb_lignes):
            for j in range(nb_colonnes):
                # coordonnées de la case i, j
                x_case = __
                y_case = __
                
                # centrons l'envahisseur dans sa case
                env = EnvahisseurFils(scene, __)
                l, h = env.dim
                x_env = x_case + __
                y_env = y_case + __              
                env.set_position(x_env, y_env)
                
                # il est temps de l'ajouter au tableau sous jacent
                self.tableau.__
        
    def supprimer_acteur(self, acteur):
        pass
    
    def deplacer(self, dx, dy):
        pass

    def tirer(self):
        pass
    
    def attaquer(self):
        pass
    
    def min_max(self):
        pass
    
    def lancer(self):
        pass
    
    # Pour s'assurer que la meute est détruite par Python
    # lorsqu'elle ne contient plus aucun envahisseur.
    def __del__(self):
        print("Mort de la meute")

if __name__ == "__main__":
    from scene import *
    from hero import Hero
    from envahisseurs import EnvahisseurMere
    m = Meute(scene, 3, 3)
    # m.deplacer(0, 100)
    # m.tirer()
    # m.attaquer()
    # m.lancer()
    EnvahisseurMere(scene)
    Hero(scene)
    del m
    fen.mainloop()
```

**Solution** pour la fin de `__init__`

```python
# ...
def __init__(self, scene, nb_lignes=3, nb_colonnes=5, largeur_case=70, hauteur_case=70):
    #...

    # remplissons les cases de la meute.
    # la position de cette meute est x=0 et y=0 (coin supérieur gauche)
    for i in range(nb_lignes):
        for j in range(nb_colonnes):
            # coordonnées de la case i, j
            x_case = j * largeur_case
            y_case = i * hauteur_case

            # centrons l'envahisseur dans sa case
            env = EnvahisseurFils(scene, self)
            l, h = env.dim
            x_env = x_case + (largeur_case-l)/2
            y_env = y_case + (hauteur_case-h)/2               
            env.set_position(x_env, y_env)

            # il est temps de l'ajouter au tableau sous jacent
            self.tableau.ajouter(env, i, j)
```

À chaque étape de l'implémentation, décommenter les parties du code de test correspondantes.

### `supprimer_acteur`, `deplacer` et `tirer`

```python
def supprimer_acteur(self, acteur):
    self.tableau.__(acteur)
    
def deplacer(self, dx, dy):
    if self.tableau.__: return

    x, y = self.pos
    self.pos = __, __
    for env in self.tableau.__:
        env.deplacer(dx, dy)

def tirer(self):
    if self.tableau.__: return

    tireurs = self.tableau.__
    tireur = choice(tireurs)
    tireur.__
```

**Solution**

```python
def supprimer_acteur(self, acteur):
    self.tableau.supprimer_valeur(acteur)
    
def deplacer(self, dx, dy):
    if self.tableau.est_vide(): return

    x, y = self.pos
    self.pos = x + dx, y + dy
    for env in self.tableau.liste_valeurs():
        env.deplacer(dx, dy)

def tirer(self):
    if self.tableau.est_vide(): return

    tireurs = self.tableau.frontiere_basse()
    tireur = choice(tireurs)
    tireur.tirer()
```

### `attaquer`

```python
def attaquer(self):
    """engage le tir de la meute à répétition
    avec un intervalle de temps aléatoire """
    if self.tableau.__: return
    self.__
    self.__.after(randint(400, 1000), __)
```

**Solution**

```python
def attaquer(self):
    """engage le tir de la meute à répétition
    avec un intervalle de temps aléatoire """
    if self.tableau.est_vide(): return
    self.tirer()
    self.scene.after(randint(400, 1000), self.attaquer)
```

### `lancer`

```python
def lancer(self):
    if __: return
    
    xmin, ymin = self.pos
    # pour la meute
    xmax = xmin + __ * self.largeur_case
    
    # si le déplacement fait sortir de la scène
    if __ + self.pas < 0 or xmax + __ > int(self.scene["width"]):
        # déplacer vers le bas d'une case
        self.deplacer(__)
        # ... et changer de direction
        self.pas = - __
    else:
        self.deplacer(__)
    
    # relancer
    __.after(__, self.lancer)
     
```

**Solution**

```python
def lancer(self):
    if self.tableau.est_vide(): return
    
    xmin, ymin = self.pos
    xmax = xmin + self.nb_colonnes * self.largeur_case
    
    # si le déplacement fait sortir de la scène
    if xmin + self.pas < 0 or xmax + self.pas > int(self.scene["width"]):
        # déplacer vers le bas ...
        self.deplacer(0, self.hauteur_case)
        # ... et changer de direction
        self.pas = - self.pas
    else:
        self.deplacer(self.pas, 0)
    
    # relancer
    self.scene.after(self.delai, self.lancer)
     
```

Arriver à ce stade, en exécutant le fichier *meute.py*, vous devriez avoir un bon aperçu du jeu.

## Conclusion

À partir de là, si vous avez bien suivi, vous disposez de toutes les notions clés pour concevoir ou cloner un jeu 2D type Arcade avec tkinter:

- la connaissance suffisante du widget `Canvas`,
- l'organisation **modulaire** du code en fichiers,
- la **séparation** entre *définition* d'une part, *test* d'autre part,
- la gestion de nombreux objets grâce à la notion de **classe**,
- la *réutilisation* et/ou *spécialisation* du code grâce à la notion d'**héritage**,
- l'organisation des objets en grille grâce à la notion de **tableau**.

Notamment, vous devriez pouvoir comprendre comment le code du *space_invaders_final* situé dans le répertoire *archives* est organisé.

Il est temps maintenant de vous faire la main sur un projet de votre cru et ... de *voler de vos propres ailes!*

## Complément pour une meilleur gestion du clavier

À faire...